In [10]:
import os
import pandas as pd
from autoemulate.core.compare import AutoEmulate
import torch
figsize = (9, 5)

In [11]:
param_filename = 'parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas'

n_samples = 2048

simulation_out_path = f'../outputs/simulations/output_{n_samples}_samples_{param_filename}/'

parameters_json_file = os.path.join(simulation_out_path, 'parameters.json')

# Load the simulation input parameters
input_params = pd.read_csv(os.path.join(simulation_out_path, f'input_samples_{n_samples}.csv'))

# Load the summary statistics
summary_stats = pd.read_csv(os.path.join(simulation_out_path, f'simulations_summary.csv'))

In [12]:
# Get column names for parameters up to 'lv.k_pas'
parameter_names = list(input_params.columns[:input_params.columns.get_loc('lv.k_pas') + 1])

input_params[parameter_names]

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,la.k_pas,lv.E_pas,lv.E_act,lv.k_pas
0,305.576111,0.364705,1047.778645,2.796825,7.379755,116.738799,5.767604,3.872886,0.253016,0.650535,0.027086,1.247329,3.465806,0.025985
1,224.490389,0.267449,1391.318553,3.568193,9.100033,91.786620,6.305379,4.954608,0.372932,0.446568,0.008090,0.806839,2.924290,0.006675
2,254.249478,0.327790,906.121586,2.608205,10.275423,104.035536,5.166425,4.290832,0.334445,0.542466,0.017685,0.985828,3.109324,0.020852
3,271.680109,0.230533,1251.458715,3.003151,8.699872,129.094902,6.925526,3.321711,0.286747,0.613231,0.020454,1.052777,2.523997,0.013266
4,285.041178,0.313907,1283.209757,3.239476,7.965886,122.133654,4.714354,4.508952,0.315288,0.459171,0.014723,1.100807,3.203127,0.011534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,285.121142,0.230828,887.757827,3.491993,10.294631,88.931860,4.651514,4.600824,0.247895,0.525992,0.009352,0.808730,2.320932,0.018579
2044,271.790319,0.313623,1066.858803,3.724367,10.729309,110.072925,6.941766,3.229777,0.349449,0.678943,0.025873,0.856512,3.002628,0.029975
2045,254.213627,0.253792,1266.494987,2.660422,7.945536,134.578851,4.950853,4.310694,0.229570,0.474975,0.012232,1.166408,2.602150,0.010709
2046,224.526976,0.350528,925.493575,3.162989,6.912402,122.320007,6.054582,4.925923,0.263373,0.512817,0.013438,1.094903,3.548719,0.023427


In [13]:
len(summary_stats.columns)

48

In [14]:
summary_stats.columns

Index(['v_ao_mean', 'v_ao_max', 'v_ao_min', 'v_art_mean', 'v_art_max',
       'v_art_min', 'v_ven_mean', 'v_ven_max', 'v_ven_min', 'v_la_mean',
       'v_la_max', 'v_la_min', 'v_lv_mean', 'v_lv_max', 'v_lv_min',
       'p_lv_mean', 'p_lv_max', 'p_lv_min', 'q_av_mean', 'q_av_max',
       'q_av_min', 'p_ao_mean', 'p_ao_max', 'p_ao_min', 'p_art_mean',
       'p_art_max', 'p_art_min', 'q_ao_mean', 'q_ao_max', 'q_ao_min',
       'p_ven_mean', 'p_ven_max', 'p_ven_min', 'q_art_mean', 'q_art_max',
       'q_art_min', 'p_la_mean', 'p_la_max', 'p_la_min', 'q_ven_mean',
       'q_ven_max', 'q_ven_min', 'q_mv_mean', 'q_mv_max', 'q_mv_min', 'T_mean',
       'T_max', 'T_min'],
      dtype='object')

In [15]:
# Turn x into a pytorch tensor
x = torch.tensor(input_params[parameter_names].values, dtype=torch.float32)

for i_output in summary_stats.columns:
    
    print(f'Processing {i_output}')

    # Create a directory for the emulators
    emulators_path = os.path.join(simulation_out_path, 'emulators', i_output)
    os.makedirs(emulators_path, exist_ok=True)

    # Get the target variable
    Y = summary_stats[i_output].values
    y = torch.tensor(Y, dtype=torch.float32)

    print(x.shape, y.shape)

    # Initialize the AutoEmulate class
    ae = AutoEmulate(x = x,
                     y = y,
                     models=["GP"])

    print(ae.summarise())

    best = ae.best_result()

    print(f'Best model is: {best.model_name}')

    # Save the best model
    print('Saving the best model...')
    ae.save(model_obj=best,
            path=emulators_path,
            use_timestamp=False)
    


Processing v_ao_mean
torch.Size([2048, 14]) torch.Size([2048])


Comparing models: 100%|██████████| 1.00/1.00 [03:22<00:00, 202s/model]


        model_name              x_transforms              y_transforms  \
0  GaussianProcess  [StandardizeTransform()]  [StandardizeTransform()]   

                                              params  rmse_test   r2_test  \
0  {'mean_module_fn': <function zero_mean at 0x31...   0.488507  0.999743   

   r2_test_std  r2_train  r2_train_std  
0     0.000029  0.999864      0.000007  
Best model is: GaussianProcess
Saving the best model...
Processing v_ao_max
torch.Size([2048, 14]) torch.Size([2048])


Comparing models: 100%|██████████| 1.00/1.00 [05:47<00:00, 348s/model]


        model_name              x_transforms              y_transforms  \
0  GaussianProcess  [StandardizeTransform()]  [StandardizeTransform()]   

                                              params  rmse_test   r2_test  \
0  {'mean_module_fn': <function zero_mean at 0x31...   0.605423  0.999795   

   r2_test_std  r2_train  r2_train_std  
0     0.000021  0.999882      0.000006  
Best model is: GaussianProcess
Saving the best model...
Processing v_ao_min
torch.Size([2048, 14]) torch.Size([2048])


...

        model_name              x_transforms              y_transforms  \
0  GaussianProcess  [StandardizeTransform()]  [StandardizeTransform()]   

                                              params  rmse_test   r2_test  \
0  {'mean_module_fn': <function linear_mean at 0x...   15.40832  0.960795   

   r2_test_std  r2_train  r2_train_std  
0     0.003214  0.969246      0.001289  
Best model is: GaussianProcess
Saving the best model...
Processing T_max
torch.Size([2048, 14]) torch.Size([2048])


Comparing models: 100%|██████████| 1.00/1.00 [01:48<00:00, 109s/model]


        model_name              x_transforms              y_transforms  \
0  GaussianProcess  [StandardizeTransform()]  [StandardizeTransform()]   

                                              params  rmse_test   r2_test  \
0  {'mean_module_fn': <function linear_mean at 0x...  15.578686  0.959259   

   r2_test_std  r2_train  r2_train_std  
0     0.003177  0.970318      0.001251  
Best model is: GaussianProcess
Saving the best model...
Processing T_min
torch.Size([2048, 14]) torch.Size([2048])


Comparing models: 100%|██████████| 1.00/1.00 [01:57<00:00, 117s/model]

        model_name              x_transforms              y_transforms  \
0  GaussianProcess  [StandardizeTransform()]  [StandardizeTransform()]   

                                              params  rmse_test  r2_test  \
0  {'mean_module_fn': <function linear_mean at 0x...  15.656792   0.9575   

   r2_test_std  r2_train  r2_train_std  
0       0.0038  0.975963      0.001054  
Best model is: GaussianProcess
Saving the best model...
